In [ ]:
import os
from pathlib import Path
from datetime import datetime
from PIL import Image
import csv, re, shutil, subprocess, sys
from typing import List, Tuple
# 偵測是否在 Colab
def in_colab():
    try:
        import google.colab
        return True
    except ImportError:
        return False

if in_colab():
    # 如果在 Colab，掛載 Google Drive
    from google.colab import drive
    drive.mount("/content/drive")

    drive_root = Path("/content/drive/MyDrive")
    PROJECT_DIR = drive_root / "CVPDL_HW1"
    print("Running on Colab, using Google Drive path:", PROJECT_DIR)

    #update this repo
    #subprocess.run(["git", "pull"], cwd=PROJECT_DIR)


else:
    # 如果在本地端
    PROJECT_DIR = Path(os.getcwd())
    print("Running locally, using local path:", PROJECT_DIR)

# 範例：自動設定常用檔案路徑
DATA_YAML = PROJECT_DIR / "yolov9_data.yaml"
WEIGHTS   = PROJECT_DIR / "weights" / "yolov9-s.pt"
PROJECT   = PROJECT_DIR / "runs" / "train"

print("DATA_YAML:", DATA_YAML)
print("WEIGHTS:", WEIGHTS)
print("PROJECT:", PROJECT)

Running locally, using local path: /Users/luweiren/Documents/Projects/CVPDL/CVPDL_HW1
DATA_YAML: /Users/luweiren/Documents/Projects/CVPDL/CVPDL_HW1/yolov9_data.yaml
WEIGHTS: /Users/luweiren/Documents/Projects/CVPDL/CVPDL_HW1/weights/yolov9-s.pt
PROJECT: /Users/luweiren/Documents/Projects/CVPDL/CVPDL_HW1/runs/train


In [2]:
#clone yolov9

In [3]:

# ========== 2️⃣ 設定子資料夾 ==========
DATASET = PROJECT_DIR / "taica-cvpdl-2025-hw-1"
TRAIN_IMG_DIR = DATASET / "train" / "img"
TEST_IMG_DIR  = DATASET / "test" / "img"
GT_FILE = DATASET / "train" / "gt.txt"

LABELS_DIR = DATASET / "train" / "labels"
YAML_PATH  = PROJECT_DIR / "yolov9_data.yaml"
REPO_DIR   = PROJECT_DIR / "yolov9"
WEIGHTS_DIR = PROJECT_DIR / "weights"
SUBMISSION_DIR = PROJECT_DIR / "submission"
CFG_DIR=PROJECT_DIR / "cfg"

for d in [LABELS_DIR, WEIGHTS_DIR, SUBMISSION_DIR]:
    d.mkdir(parents=True, exist_ok=True)

In [4]:
if not REPO_DIR.exists():
    print("📦 Cloning YOLOv9 repository...")
    subprocess.run(["git", "clone", "https://github.com/WongKinYiu/yolov9.git", str(REPO_DIR)], check=True)

print("📦 Installing dependencies...")
subprocess.run([sys.executable, "-m", "pip", "install", "-r", "requirements.txt"], cwd=REPO_DIR)

📦 Installing dependencies...
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com


CompletedProcess(args=['/Users/luweiren/miniforge3/envs/PytorchLearning/bin/python', '-m', 'pip', 'install', '-r', 'requirements.txt'], returncode=0)

In [5]:
# ========== 4️⃣ 基本工具函式 ==========
def list_images(img_dir: Path) -> List[Path]:
    return sorted(list(img_dir.glob("*.jpg")) + list(img_dir.glob("*.png")))

def im_size(path: Path) -> Tuple[int, int]:
    with Image.open(path) as im:
        return im.size

def to_int_id(name_or_id: str) -> int:
    s = re.sub(r"^0+", "", name_or_id.replace(".jpg","").replace(".png",""))
    return int(s) if s else 0

def parse_gt_line(line: str):
    """
    解析 gt.txt 的一行
    格式: image_id,left,top,width,height (可能有多個 box)
    返回: (image_id, [(left, top, width, height), ...])
    """
    toks = re.split(r"[,\s]+", line.strip())
    if len(toks) < 5: 
        return "", []
    head = toks[0]  # image_id
    rest = toks[1:]  # 剩下的是坐標
    boxes = []
    for i in range(0, len(rest), 4):  # 每 4 個一組 (left, top, width, height)
        if i + 3 < len(rest):
            try:
                left = float(rest[i])
                top = float(rest[i+1])
                width = float(rest[i+2])
                height = float(rest[i+3])
                boxes.append((left, top, width, height))
            except:
                pass
    return head, boxes

def box_to_yolo_format(left: float, top: float, width: float, height: float, 
                       img_width: int, img_height: int, class_id: int = 0) -> str:
    """
    將 bounding box 從 (left, top, width, height) 像素格式
    轉換為 YOLO 格式: class center_x center_y width height (歸一化)
    
    參數:
        left: 左上角 x 坐標 (像素)
        top: 左上角 y 坐標 (像素)
        width: 寬度 (像素)
        height: 高度 (像素)
        img_width: 圖片寬度
        img_height: 圖片高度
        class_id: 類別 ID (預設 0)
    
    返回:
        YOLO 格式字串: "class cx cy w h"
    """
    # 計算中心點坐標
    center_x = (left + width / 2.0) / img_width
    center_y = (top + height / 2.0) / img_height
    
    # 歸一化寬高
    norm_width = width / img_width
    norm_height = height / img_height
    
    # 限制在 [0, 1] 範圍內
    center_x = max(0.0, min(1.0, center_x))
    center_y = max(0.0, min(1.0, center_y))
    norm_width = max(0.0, min(1.0, norm_width))
    norm_height = max(0.0, min(1.0, norm_height))
    
    return f"{class_id} {center_x:.6f} {center_y:.6f} {norm_width:.6f} {norm_height:.6f}"

# 測試轉換函數
print("📝 Box to YOLO format conversion example:")
print("Input: left=100, top=50, width=200, height=150 (in 640x480 image)")
result = box_to_yolo_format(100, 50, 200, 150, 640, 480, 0)
print(f"Output: {result}")
print("Explanation: class=0, center_x=(100+200/2)/640=0.3125, center_y=(50+150/2)/480=0.2604, w=200/640=0.3125, h=150/480=0.3125")

📝 Box to YOLO format conversion example:
Input: left=100, top=50, width=200, height=150 (in 640x480 image)
Output: 0 0.312500 0.260417 0.312500 0.312500
Explanation: class=0, center_x=(100+200/2)/640=0.3125, center_y=(50+150/2)/480=0.2604, w=200/640=0.3125, h=150/480=0.3125


In [6]:
# ========== 5️⃣ 將 gt.txt 轉成 YOLO 標籤 ==========
def convert_gt_to_yolo_labels():
    """
    將 gt.txt 中的 bounding boxes 轉換為 YOLO 格式
    gt.txt 格式: image_id,left,top,width,height
    YOLO 格式: class center_x center_y width height (歸一化到 0-1)
    """
    imgs = {p.name: p for p in list_images(TRAIN_IMG_DIR)}
    converted_count = 0
    skipped_count = 0
    
    with open(GT_FILE, "r") as f:
        for line_num, line in enumerate(f, 1):
            name_or_id, boxes = parse_gt_line(line)
            if not boxes: 
                continue
            
            # 將 image_id 轉換為檔名格式
            key = f"{to_int_id(name_or_id):08d}.jpg"
            img_path = imgs.get(key)
            
            if(img_path is None):
                skipped_count += 1
                continue
            # 獲取圖片尺寸
            img_width, img_height = im_size(img_path)
            
            
            # 轉換每個 box 到 YOLO 格式
            yolo_lines = []
            for left, top, width, height in boxes:
                yolo_line = box_to_yolo_format(left, top, width, height, 
                                               img_width, img_height, 
                                               class_id=0)  # 單類別，class_id=0
                yolo_lines.append(yolo_line)
            
            # 寫入 YOLO 標籤檔案
            label_file = LABELS_DIR / f"{Path(key).stem}.txt"
            if(label_file.exists()):
                with open(label_file, "a", encoding="utf-8") as f:
                    f.write("\n".join(yolo_lines) + "\n")
            else:
                with open(label_file, "w", encoding="utf-8") as f:
                    f.write("\n".join(yolo_lines) + "\n")
            converted_count += 1
    
    print(f"✅ YOLO labels saved at: {LABELS_DIR}")
    print(f"📊 Converted: {converted_count} images")
    print(f"⚠️  Skipped: {skipped_count} images (not found)")

#clear labels
for label_file in LABELS_DIR.glob("*.txt"):
    label_file.unlink()

convert_gt_to_yolo_labels()

✅ YOLO labels saved at: /Users/luweiren/Documents/Projects/CVPDL/CVPDL_HW1/taica-cvpdl-2025-hw-1/train/labels
📊 Converted: 38619 images
⚠️  Skipped: 128 images (not found)


In [7]:
# ========== 6️⃣ 寫入 data.yaml ==========
YAML_PATH.write_text(f"""# YOLOv9 data file
train: {TRAIN_IMG_DIR.as_posix()}
val: {TRAIN_IMG_DIR.as_posix()}
test: {TEST_IMG_DIR.as_posix()}
nc: 1
names: ['object']
""")
print(f"✅ Wrote YAML to {YAML_PATH}")


✅ Wrote YAML to /Users/luweiren/Documents/Projects/CVPDL/CVPDL_HW1/yolov9_data.yaml


In [11]:
# ========== 7️⃣ 訓練設定 ==========
# 選擇權重（建議 yolov9-c.pt 或 yolov9-e.pt）
#!wget -nc https://github.com/WongKinYiu/yolov9/releases/download/v1.0/yolov9-s.pt -P {WEIGHTS_DIR}

EPOCHS = 50
BATCH  = 8
IMGSZ  = 640
DEVICE = "cpu" if not in_colab() else 0   # GPU

# 開始訓練
train_cmd = [
    sys.executable, str(REPO_DIR / "train.py"),
    "--data", str(YAML_PATH),
    "--weights", str(WEIGHTS_DIR / "yolov9-s.pt"),
    "--img", str(IMGSZ),
    "--epochs", str(EPOCHS),
    "--batch", str(BATCH),
    "--device", str(DEVICE),
    "--project", "runs/train",
    "--name", "cvpdl_hw1",
    "--cfg", str(CFG_DIR / "yolov9-s.yaml"),
    "--hyp", str(REPO_DIR / "data/hyps/hyp.scratch-high.yaml"),
    "--exist-ok"
]
subprocess.run(train_cmd,cwd=str(PROJECT_DIR), check=True)

# ========== 8️⃣ 對 test 集推論 ==========
best_weight = sorted((REPO_DIR / "runs/train").rglob("best.pt"))[-1]
detect_cmd = [
    "python", "detect.py",
    "--weights", str(best_weight),
    "--source", str(TEST_IMG_DIR),
    "--img", str(IMGSZ),
    "--conf", "0.001",
    "--iou", "0.6",
    "--save-txt", "--save-conf",
    "--project", "runs/detect",
    "--name", "cvpdl_hw1_test",
    "--exist-ok"
]
subprocess.run(detect_cmd, cwd=REPO_DIR, check=True)
print("✅ Detection finished.")

# ========== 9️⃣ 產生 Submission CSV ==========
labels_dir = sorted((REPO_DIR / "runs/detect").glob("cvpdl_hw1_test*"))[-1] / "labels"
out_csv = SUBMISSION_DIR / f"submission_{datetime.now().strftime('%Y%m%d_%H%M%S')}.csv"

def normalized_to_pixels(cx, cy, w, h, W, H):
    return (cx - w/2)*W, (cy - h/2)*H, w*W, h*H

with open(out_csv, "w", newline="", encoding="utf-8") as f:
    writer = csv.writer(f)
    writer.writerow(["Image_ID", "PredictionString"])
    for img_path in list_images(TEST_IMG_DIR):
        W,H = im_size(img_path)
        stem = img_path.stem
        img_id = to_int_id(stem)
        lbl_path = labels_dir / f"{stem}.txt"
        pred_str = ""
        if lbl_path.exists():
            lines = lbl_path.read_text().strip().splitlines()
            preds = []
            for line in lines:
                c,conf,cx,cy,w,h = map(float,line.strip().split())
                preds.append((c,conf,cx,cy,w,h))
            preds.sort(key=lambda t:t[1], reverse=True)
            parts=[]
            for c,conf,cx,cy,w,h in preds:
                x,y,ww,hh = normalized_to_pixels(cx,cy,w,h,W,H)
                parts += [f"{conf:.6f}",f"{x:.2f}",f"{y:.2f}",f"{ww:.2f}",f"{hh:.2f}",f"{int(c)}"]
            pred_str=" ".join(parts)
        writer.writerow([img_id,pred_str])

print(f"✅ Submission file saved: {out_csv}")


train: weights=/Users/luweiren/Documents/Projects/CVPDL/CVPDL_HW1/weights/yolov9-s.pt, cfg=/Users/luweiren/Documents/Projects/CVPDL/CVPDL_HW1/cfg/yolov9-s.yaml, data=/Users/luweiren/Documents/Projects/CVPDL/CVPDL_HW1/yolov9_data.yaml, hyp=/Users/luweiren/Documents/Projects/CVPDL/CVPDL_HW1/yolov9/data/hyps/hyp.scratch-high.yaml, epochs=50, batch_size=8, imgsz=640, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, noplots=False, evolve=None, bucket=, cache=None, image_weights=False, device=cpu, multi_scale=False, single_cls=False, optimizer=SGD, sync_bn=False, workers=8, project=runs/train, name=cvpdl_hw1, exist_ok=True, quad=False, cos_lr=False, flat_cos_lr=False, fixed_lr=False, label_smoothing=0.0, patience=100, freeze=[0], save_period=-1, seed=0, local_rank=-1, min_items=0, close_mosaic=0, entity=None, upload_dataset=False, bbox_interval=-1, artifact_alias=latest
YOLO 🚀 v0.1-104-g5b1ea9a Python-3.10.4 torch-1.13.0.dev20220608 CPU

hyperparameters: lr0=0.01, lrf

CalledProcessError: Command '['/Users/luweiren/miniforge3/envs/PytorchLearning/bin/python', '/Users/luweiren/Documents/Projects/CVPDL/CVPDL_HW1/yolov9/train.py', '--data', '/Users/luweiren/Documents/Projects/CVPDL/CVPDL_HW1/yolov9_data.yaml', '--weights', '/Users/luweiren/Documents/Projects/CVPDL/CVPDL_HW1/weights/yolov9-s.pt', '--img', '640', '--epochs', '50', '--batch', '8', '--device', 'cpu', '--project', 'runs/train', '--name', 'cvpdl_hw1', '--cfg', '/Users/luweiren/Documents/Projects/CVPDL/CVPDL_HW1/cfg/yolov9-s.yaml', '--hyp', '/Users/luweiren/Documents/Projects/CVPDL/CVPDL_HW1/yolov9/data/hyps/hyp.scratch-high.yaml', '--exist-ok']' returned non-zero exit status 1.